<td>
   <a target="_blank" href="https://labelbox.com" ><img src="https://labelbox.com/blog/content/images/2021/02/logo-v4.svg" width=256/></a>
</td>

<td>
<a href="https://colab.research.google.com/github/Labelbox/labelbox-python/blob/develop/examples/basics/batches.ipynb" target="_blank"><img
src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"></a>
</td>

<td>
<a href="https://github.com/Labelbox/labelbox-python/blob/develop/examples/basics/batches.ipynb" target="_blank"><img
src="https://img.shields.io/badge/GitHub-100000?logo=github&logoColor=white" alt="GitHub"></a>
</td>

## Batches (*Currently in Public Beta*)

* A Batch is collection of datarows picked out of a Data Set.
* A Datarow cannot be part of more than one batch in a project.
* Batches work for all data types, but there should only be one data type per batch.
* Batches may not be shared between projects.
* Batches may have Datarows from multiple Datasets.
* Datarows can only be attached to a Project as part of a single Batch.
* You can set priority for each Batch.

In [1]:
!pip install labelbox[data]

zsh:1: no matches found: labelbox[data]


In [ ]:
from labelbox import DataRow, Client
from labelbox.schema.queue_mode import QueueMode
import random

Set the following cell with your data to run this notebook:

In [ ]:
PROJECT_NAME = "Batch Queue Demo"  #text project
DATASET_NAME = "Batch Queue Demo Data"

# API Key and Client
Provide a valid api key below in order to properly connect to the Labelbox Client.

In [ ]:
# Add your api key
API_KEY = None
client = Client(api_key=API_KEY)

In [ ]:
dataset = client.create_dataset(name=DATASET_NAME)

uploads = []
for i in range(10):
    uploads.append({
        'external_id': i,
        'row_data': 'https://picsum.photos/200/300'
    })
dataset.create_data_rows(uploads)

# Ensure project is in batch mode:

In [ ]:
project = client.create_project(name=PROJECT_NAME, queue_mode=QueueMode.Batch)

# Collect Datarow id's:

### Select All Data Rows from dataset.

In [ ]:
data_row_ids = [dr.uid for dr in dataset.export_data_rows()]

### Randomly sample

Rather than selecting all of the data we sample 5 data rows at random

In [ ]:
sample = random.sample(data_rows, 5)

# Batch Manipulation

### Create a Batch:

In [ ]:
batch = project.create_batch(
  "first batch", # Each batch in a project must have a unique name
  sample, # A list of data rows or data row ids
  5 # priority between 1(Highest) - 5(lowest)
)

In [ ]:
# number of data rows in the batch
batch.size

### List DataRows in a Batch
Note: You can view your batch through in the *Data Row tab* of the project

In [ ]:
data_rows = [dr for dr in batch.export_data_rows()]

### Remove queued data rows by batch (Not supported yet)
Note: You can do this through the batch management pane on the data rows tab of the project